In [1]:
#!/usr/bin/env python3
#pip install  rioxarray==0.3.1
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import rasterio
import os
import matplotlib.colors
scriptsdir = os.getcwd()
from scipy.interpolate import griddata
from functools import reduce
import xarray
import itertools
import argparse
import matplotlib.colors as mcolors

In [2]:
taxas=["Mammals","Amphibians","Bird"]
models=["GAM","GBM"]

In [3]:
species_names =  {}
for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names[taxa] = available_names[:10]
    

species_list= []
species_list.append(species_names[taxa])
# Load the df_forest and df_non_forest DataFrames
df_forest = pd.read_csv('/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_forest[35].csv')
df_non_forest = pd.read_csv('/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_non_forest[35].csv')

# Create a dictionary to store the species category (forest or non-forest)
species_category = {}

# Iterate over the forest and non-forest categories
for category, df in {'forest': df_forest, 'non_forest': df_non_forest}.items():
    # Create a list to store species names based on the category
    category_species = list(df['Species'])


    model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
             '2052', '2056', '2080', '2100', '2150', '2200', '2250']

    for species_name in category_species:
        # Check if the species is in the df_forest DataFrame
        if species_name in df_forest['Species'].values:
            species_category[species_name] = 'forest'
        # Check if the species is in the df_non_forest DataFrame
        elif species_name in df_non_forest['Species'].values:
            species_category[species_name] = 'non_forest'


    def newvalue_fun_forest(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {species_name: {} for species_name in category_species} for model_name in model_names}
        sum_bin_dict = {model_name: {species_name: {} for species_name in category_species} for model_name in model_names}


        for model_name in model_names:
            for species_name in df_non_forest['Species']:
                found = False
                for taxa in taxas:
                    try:
                        if is_historical:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                        else:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time, model_name, scenario), decode_times=False)
                        found = True
                        break  # Exit the loop if species is found in any taxa
                    except:
                        continue
                if found:
                    newvalue = ds["newvalue"]
                    sum_bin = ds["sum_bin"]

                    newvalue_dict[model_name][species_name] = newvalue
                    sum_bin_dict[model_name][species_name] = sum_bin
                    print(f"Species {species_name} found in {taxa}.")



            for species_name in df_forest['Species']:
                species_name_cleaned = species_name.strip()  # Remove leading and trailing spaces
                found = False
                for taxa in taxas:
                    try:
                        if is_historical:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                        else:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time, model_name, scenario), decode_times=False)
                        found = True
                        break  # Exit the loop if species is found in any taxa
                    except:
                        continue
                if found:
                    newvalue = ds["newvalue"]
                    sum_bin = ds["sum_bin"]

                    newvalue_dict[model_name][species_name] = newvalue
                    sum_bin_dict[model_name][species_name] = sum_bin
                    print(f"Species {species_name} found in {taxa}.")
                else:
                    print(f"Species {species_name} not found in any taxa")

        projections_dict = {}
        for species_name in category_species:
            value_list = []
            for model_name in model_names:
                value_bin = newvalue_dict[model_name][species_name]
                value_list.append(value_bin)
                
            value_bin_concat = xr.concat(value_list, dim="model_name")
            mean_value_bin = value_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_value_bin
            
            value_bin_list = list(projections_dict.values())
            mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
            mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)

        return mean_value_bin


FileNotFoundError: [Errno 2] No such file or directory: '/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_forest[35].csv'

In [208]:
mean_value_bin_hist

{'GFDL-ESM2M': {'Sorex_camtschatica': <xarray.DataArray 'newvalue' (lat: 360, lon: 720)>
  [259200 values with dtype=float64]
  Coordinates:
    * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
    * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
  'Axis_porcinus': <xarray.DataArray 'newvalue' (lat: 360, lon: 720)>
  [259200 values with dtype=float64]
  Coordinates:
    * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
    * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
  'Phyllotis_amicus': <xarray.DataArray 'newvalue' (lat: 360, lon: 720)>
  [259200 values with dtype=float64]
  Coordinates:
    * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
    * lat      (lat) float64 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75,
  'Crinia_bilingua': <xarray.DataArray 'newvalue' (lat: 360, lon: 720)>
  [259200 values with dtype=float64]
  Coordinates:
    * lon     

In [205]:
mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)

Species Sorex_camtschatica found in Mammals.
Species Axis_porcinus found in Mammals.
Species Phyllotis_amicus found in Mammals.
Species Crinia_bilingua found in Amphibians.
Species Rhinolophus_euryotis found in Mammals.
Species Hypsugo_kitcheneri found in Mammals.
Species Bdeogale_nigripes found in Mammals.
Species Hystrix_javanica found in Mammals.
Species Spilogale_putorius found in Mammals.
Species Cheirogaleus_crossleyi found in Mammals.
Species Hydrochoerus_hydrochaeris found in Mammals.
Species Hyloscirtus_palmeri found in Amphibians.
Species Rhinella_dapsilis found in Amphibians.
Species Smilisca_phaeota found in Amphibians.
Species Scinax_cardosoi found in Amphibians.
Species Amolops_larutensis found in Amphibians.
Species Allobates_talamancae found in Amphibians.
Species Litoria_peronii found in Amphibians.
Species Boophis_andohahela found in Amphibians.
Species Dendropsophus_seniculus found in Amphibians.
Species Lophornis_ornatus found in Bird.
Species Eugerygone_rubra found

In [155]:
species_names =  {}
for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names[taxa] = available_names[:10]
    

species_list= []
species_list.append(species_names[taxa])
# Load the df_forest and df_non_forest DataFrames
df_forest = pd.read_csv('/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_forest[35].csv')
df_non_forest = pd.read_csv('/storage/homefs/ch21o450/scripts/BioScenComb/habitat_counts/df_non_forest[35].csv')

# Create a dictionary to store the species category (forest or non-forest)
species_category = {}

# Iterate over the forest and non-forest categories
for category, df in {'forest': df_forest, 'non_forest': df_non_forest}.items():
    # Create a list to store species names based on the category
    category_species = list(df['Species'])


    model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
    years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
             '2052', '2056', '2080', '2100', '2150', '2200', '2250']

    for species_name in category_species:
        # Check if the species is in the df_forest DataFrame
        if species_name in df_forest['Species'].values:
            species_category[species_name] = 'forest'
        # Check if the species is in the df_non_forest DataFrame
        elif species_name in df_non_forest['Species'].values:
            species_category[species_name] = 'non_forest'


    def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {species_name: {} for species_name in category_species} for model_name in model_names}
        sum_bin_dict = {model_name: {species_name: {} for species_name in category_species} for model_name in model_names}


        for model_name in model_names:
            for species_name in df_non_forest['Species']:
                found = False
                for taxa in taxas:
                    try:
                        if is_historical:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                        else:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time, model_name, scenario), decode_times=False)
                        found = True
                        break  # Exit the loop if species is found in any taxa
                    except:
                        continue
                if found:
                    print(f"Species {species_name} found in {taxa}.")
                    projections_dict = {}

                    value_list = []
                    for model_name in model_names:
                        value_bin = newvalue_dict[model_name][species_name]
                        value_list.append(value_bin)
                        value_bin_concat = xr.concat(value_list, dim="model_name")
                        mean_value_bin = value_bin_concat.mean(dim="model_name")
                        projections_dict[species_name] = mean_value_bin


            for species_name in df_forest['Species']:
                species_name_cleaned = species_name.strip()  # Remove leading and trailing spaces
                found = False
                for taxa in taxas:
                    try:
                        if is_historical:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                        else:
                            ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time, model_name, scenario), decode_times=False)
                        found = True
                        break  # Exit the loop if species is found in any taxa
                    except:
                        continue
                if found:
                    newvalue = ds["newvalue"]
                    sum_bin = ds["sum_bin"]

                    newvalue_dict[model_name][species_name] = newvalue
                    sum_bin_dict[model_name][species_name] = sum_bin
                    print(f"Species {species_name} found in {taxa}.")
                else:
                    print(f"Species {species_name} not found in any taxa")
                    projections_dict = {}
                    for model_name in model_names:
                        value_bin = newvalue_dict[model_name][species_name]
                        value_list.append(value_bin)
                        value_bin_concat = xr.concat(value_list, dim="model_name")
                        mean_value_bin = value_bin_concat.mean(dim="model_name")
                        projections_dict[species_name] = mean_value_bin


        mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
        return mean_value_bin




    def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}
        lu_sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for name in category_species:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, name, time, model_name, scenario), decode_times=False)
                sum_bin = ds["sum_bin"]

                sum_bin_dict[model_name][species_name] = sum_bin
                

        projections_dict = {}

        for name in cacategory_species:
            sum_bin_list = []
            for model_name in model_names:
                sum_bin = sum_bin_dict[model_name][name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[name] = mean_sum_bin

        mean_sum_bin_list = list(projections_dict.values())
        mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin



In [162]:
value_list

NameError: name 'value_list' is not defined

In [156]:
mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)

Species Sorex_camtschatica found in Mammals.


TypeError: can only concatenate xarray Dataset and DataArray objects, got <class 'dict'>

In [115]:
taxas

['Mammals', 'Amphibians']

In [94]:
mean_value_bin_future = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)

Species Sorex_camtschatica not found in any taxa
Species Axis_porcinus not found in any taxa
Species Phyllotis_amicus not found in any taxa
Species Crinia_bilingua not found in any taxa
Species Muscicapa_olivascens not found in any taxa
Species Psittacula_krameri not found in any taxa
Species Sorex_camtschatica not found in any taxa
Species Axis_porcinus not found in any taxa
Species Phyllotis_amicus not found in any taxa
Species Crinia_bilingua not found in any taxa
Species Muscicapa_olivascens not found in any taxa
Species Psittacula_krameri not found in any taxa
Species Sorex_camtschatica not found in any taxa
Species Axis_porcinus not found in any taxa
Species Phyllotis_amicus not found in any taxa
Species Crinia_bilingua not found in any taxa
Species Muscicapa_olivascens not found in any taxa
Species Psittacula_krameri not found in any taxa
Species Sorex_camtschatica not found in any taxa
Species Axis_porcinus not found in any taxa
Species Phyllotis_amicus not found in any taxa
Sp

KeyError: 'Sorex_camtschatica'

In [ ]:


ap = argparse.ArgumentParser()

# collect the function arguments

ap.add_argument('-m', '--model', type=str, help="model, string", nargs="+", required=True)
ap.add_argument('-a', '--taxa', type=str, help="taxa, string", nargs="+", required=True)

# parse the arguments to the args object
args = ap.parse_args()

# *************************************************
# Get arguments
# *************************************************
print(args)

models = args.model
taxas = args.taxa
print(taxas)

model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
years = ['1845', '1990', '1995', '2009', '2010', '2020', '2026', '2032', '2048', '2050',
         '2052', '2056', '2080', '2100', '2150', '2200', '2250']

for taxa in taxas:
    for model in models:
        dir_species = "/storage/scratch/users/ch21o450/data/LandClim_Output/" + model + "/" + taxa + "/EWEMBI/"
        available_file = os.listdir(dir_species)
        available_names = [x.split("_[1146].nc")[0] for x in available_file]

    species_names = available_names

    def newvalue_fun(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)

                newvalue = ds["newvalue"]
                sum_bin = ds["sum_bin"]

                newvalue_dict[model_name][species_name] = newvalue
                sum_bin_dict[model_name][species_name] = sum_bin

        projections_dict = {}

        for species_name in species_names:
            value_list = []
            for model_name in model_names:
                value_bin = newvalue_dict[model_name][species_name]
                #value_bin = value_bin.where(value_bin > 0, 1)
                #value_bin = (value_bin > 0.00)

                value_list.append(value_bin)
            value_bin_concat = xr.concat(value_list, dim="model_name")
            mean_value_bin = value_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_value_bin

        value_bin_list = list(projections_dict.values())
        mean_value_bin = xr.concat(value_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_value_bin = mean_value_bin.where(mean_value_bin > 0, 0)
        return mean_value_bin

    def calculate_mean(time, model, netcdf_path_format, is_historical=False, scenario=None):
        newvalue_dict = {model_name: {} for model_name in model_names}
        sum_bin_dict = {model_name: {} for model_name in model_names}
        lu_sum_bin_dict = {model_name: {} for model_name in model_names}

        for model_name in model_names:
            for species_name in species_names:
                if is_historical:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, species_name, time), decode_times=False)
                else:
                    ds = xr.open_dataset(netcdf_path_format.format(model, taxa, model_name, scenario, species_name, time), decode_times=False)
                sum_bin = ds["sum_bin"]
                #lu_sum_bin = ds["sum_lu_binary"]
                #sum_bin = (sum_bin > 0.00)

                sum_bin_dict[model_name][species_name] = sum_bin
                #lu_sum_bin_dict[model_name][species_name] = lu_sum_bin

        projections_dict = {}

        for species_name in species_names:
            sum_bin_list = []
            for model_name in model_names:
                sum_bin = sum_bin_dict[model_name][species_name]
                sum_bin_list.append(sum_bin)
            sum_bin_concat = xr.concat(sum_bin_list, dim="model_name")
            mean_sum_bin = sum_bin_concat.mean(dim="model_name")
            projections_dict[species_name] = mean_sum_bin

        mean_sum_bin_list = list(projections_dict.values())
        mean_sum_bin = xr.concat(mean_sum_bin_list, dim="species").sum(dim="species")  # Ensemble mean over species
        mean_sum_bin = mean_sum_bin.where(mean_sum_bin > 0, 0)

        return mean_sum_bin

    historical_time = 1146
    future_times = [35, 65]
    scenarios = ["rcp26"]

    netcdf_path_format_future = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/{}/{}/{}_[{}].nc"
    netcdf_path_format_hist = "/storage/scratch/users/ch21o450/data/LandClim_Output/{}/{}/EWEMBI/{}_[{}].nc"

    mean_value_bin_hist = newvalue_fun(historical_time, model, netcdf_path_format_hist, is_historical=True)
    mean_sum_bin_hist = calculate_mean(historical_time, model, netcdf_path_format_hist, is_historical=True)

    mean_sum_bin_hist = mean_sum_bin_hist.isel(time=0)

    year_indices = {1146: '1995', 35: '2050', 65: '2080', 85: '2100'}

    for future_time in future_times:
        for scenario in scenarios:
            if future_time == 35 or future_time == 65:
                model_names = ['GFDL-ESM2M', 'IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']
            elif future_time == 85:
                model_names = ['IPSL-CM5A-LR', 'HadGEM2-ES', 'MIROC5']

            filename = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{historical_time}_{scenario}_species_count_per_SCDM_00th_mean_value_bin_future.nc"
            mean_value_bin_hist.to_netcdf(filename)

            filename2 = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{historical_time}_{scenario}_species_count_per_SCDM_00th_mean_sum_bin_future.nc"
            mean_sum_bin_hist.to_netcdf(filename2)

            mean_value_bin_future = newvalue_fun(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future = calculate_mean(future_time, model, netcdf_path_format_future, is_historical=False, scenario=scenario)
            mean_sum_bin_future = mean_sum_bin_future.isel(time=0)

            filename = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{future_time}_{scenario}_species_count_per_SCDM_00th_mean_value_bin_future.nc"
            mean_value_bin_future.to_netcdf(filename)

            filename2 = f"/storage/homefs/ch21o450/scripts/BioScenComb/intermediate_results/{taxa}_{model}_{future_time}_{scenario}_species_count_per_SCDM_00th_mean_sum_bin_future.nc"
            mean_sum_bin_future.to_netcdf(filename2)
